# Char-based text generation with LSTM

In [0]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [22]:
from google.colab import files

uploaded = files.upload()

Saving Память_света.txt to Память_света (1).txt


In [0]:
# import io

# data = io.StringIO(uploaded['Память света.txt'].decode('utf-8'))

# New Section

# New Section

In [24]:
TRAIN_TEXT_FILE_PATH = 'Память света.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
    text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'а', 'е', 'н', 'и', 'т', 'л', 'с', 'р', 'в', 'д', 'м', 'у', 'к', 'п', ',', 'ь', 'я', 'ы', '.', 'з', 'б', 'г', 'ч', 'й', 'ж', '—', 'х', 'ш', '\n', 'ё', 'ю', 'э', 'щ', 'О', 'Т', 'П', 'ц', 'М', 'С', 'В', 'Н', 'А', 'И', '?', 'Д', 'К', 'Р', 'Э', '-', 'Б', 'Е', 'Л', 'Г', 'Я', '…', '!', 'ф', 'Ч', 'У', 'З', '«', '»', '’', '*', ':', 'Ш', 'Х', 'Ф', 'ъ', 'Ж', 'Ю', ';', 'Ь', 'Ы', 'Й', 'Ц', 'Ё', 'Щ', 'ó', '2', '1', '3', '4', '0', '7', '5', '6', '8', '9', 'C', '\u2003', '́', 'c', '(', ')']


In [0]:
SEQ_LEN = 256
BATCH_SIZE = 50

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [0]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [0]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [33]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=5)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 10000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

RuntimeError: ignored

In [37]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=1000, 
    start_text='. '
    )
)

. 
 *  *  *
 
 — Нам нужно подобное усталость в померк. — Я не поднял все на то тому, что он поднял руку и рассказывал это станут и просто просто подняться на неё было способности по себе на поле боя на себя приказы. Солдаты подобное выстроились в ответ. Она подобно поднялась на восток продолжать прошло с поле боя с небольшей крови. Он поднял его в сторону. Это было не подобной свои выполнять странными под собой в сторону в конце тех пор, что это было не подошла и поднял вверх с поля на своим воздух. Он поднял власть с красным в поле боя с ними на передний солдат, который присоединился на себя в прошлом на месте.
 — Свет, — произнёс Мэт. — Ты всё ответил Ранд. — Я пристально поднял руку в сторону в попытки. — Он поднял вверх в сторону, но это было бы это было полезным по только больше подобного волки поднять троллоков, закричала с ними копья, подняв всех стороны по стороне. Они просто стало было не подобное командованием с ней в порядке в сторону и подняли его в поле боя от последний п